In [16]:
### simplex mean width computations
#n,a,N as in the paper, pr is precision
def mw(n,a,N,pr):
    coef=gamma(n/2)/gamma((n+1)/2)/2
    #tail estimates come from multipying bound (16) by coef.
    tail=(n+1)/(sqrt(2*pi))*exp(-a)
    print("tail error:",tail.n(pr))
    su=0 #upper sum from proposition 4
    sl=0 #lower sum from proposition 4
    for k in range(N+1):
        v = erf(k*a/N/sqrt(2)).n(pr)
        g = 1-((1+v)/2)^(n+1)-((1-v)/2)^(n+1)
        if k!=0:
            sl += g
        if k!=N:
            su += g
    eru=coef*((a/N)*su+tail)
    erl=coef*(a/N)*sl
    print("lower bound:",erl.n(pr))
    print("upper bound:",eru.n(pr))

In [17]:
mw(5,15,10000,100)

tail error: 7.3222421592652987129684264968e-7
lower bound: 0.42089568661063458489794419917
upper bound: 0.42137885439233364550302887424


In [18]:
mw(6,15,30000,100)

tail error: 8.5426158524761818317964975797e-7
lower bound: 0.40679792591232787497974051859
upper bound: 0.40694628272626943924496227155


In [19]:
mw(7,25,500000,100)

tail error: 4.4323903964606663926347770161e-11
lower bound: 0.39425341866192872475570590880
upper bound: 0.39426715778791651227186217374


In [20]:
mw(8,25,2000000,100)

tail error: 4.9864391960182496917141241431e-11
lower bound: 0.38300591777784167416143202214
upper bound: 0.38300912913766238653614144023


In [21]:
mw(9,25,2000000,100)

tail error: 5.5404879955758329907934712701e-11
lower bound: 0.37288085996328295783769721949
upper bound: 0.37288388315644269849975837596


In [22]:
mw(10,25,2000000,100)

tail error: 6.0945367951334162898728183972e-11
lower bound: 0.36370177705311049973894915373
upper bound: 0.36370463999339208406413741346


In [23]:
mw(11,25,2000000,100)

tail error: 6.6485855946909995889521655242e-11
lower bound: 0.35532488888185190759925123009
upper bound: 0.35532761375154940442725921495


In [24]:
mw(12,25,2000000,100)

tail error: 7.2026343942485828880315126512e-11
lower bound: 0.34763512908359788268469021749
upper bound: 0.34763773366699930836973861652


In [29]:
### proof of theorem 1: not necessarily symmetric body
# the procedure bonnesen generates an inequlaity (13) for appropriate i,j,k, R
def bonnesen(i,j,k,wi,wk,R):
    #here i<j<k
    return ((k-j)*(i+1)*R^i*wi+(j-i)*(k+1)*R^k*wk)/(R^j*(j+1)*(k-i))

# Array dens is best know densities of sphere covering in corresponding dimesnion, indexed by n. 
# The array is constructed according to Table 3 of the paper according to Schurmann and Vallentin  
dens = [0]*14
dens[1:5]=[1, 1.209199, 1.463505, 1.765529, 2.124286]
dens[6:13]=[2.464801, 2.900024, 3.142202, 4.340185, 5.251713 ,5.598338, 7.510113, 7.864060] #as in Schurmann and Vallentin
for i in range(14):
    dens[i] += 0.000005 #we upper bound the rounded values for the density from Schurmann and Vallentin; for A_n^* lattice exact is used below

#for n in range(5,9):
for n in range(2,13):
    print("n=",n)
    B=pi^(n/2)/gamma(n/2+1) ##Volume of the ball B_2^n
    den=dens[n]
    if n in [2,3,4,5,10,12]:
        den = sqrt(n+1)*(n*(n+2)/12/(n+1))^(n/2)*B #exact value for density A_n^* lattice
    print("covering density", den.n())
    S=n^(n/2)*(n+1)^((n+1)/2)/factorial(n) #simplex volume W_0(\Delta^n)
    # Mean width of a simplex calculations 

    mwa=[0,0,
        0.954929658551372/2,#n=2 from Finch
       0.912260171954089/2,#n=3 from Finch
        0.874843256085440/2,#n=4 from Finch
       0.4215,#n=5
       0.407,#n=6
        0.39427,#n=7
        0.38301,#n=8
       0.3727, #n=9
       0.363, #n=10
        0.355, #n=11
        0.347 #n=12
        ]
    # w is a vector of wi uperbounds, we use initial estimate W_i\leq W_0, W_n=B 

    w = [S for i in range(n)]+[B]
    w[n-1]=B*mwa[n]*sqrt(2*n*(n+1))#the last factor to properly rescale the simplex, the bouns is according to (20)
    # a is the upperbound acoording to (18)
    a = (den/B)*sum(binomial(n,t)*w[t] for t in range(0,n+1))
    print("starting bound",a.n())

    ###these are sufficient for the result
    ijka=[[],[],[],[],[],
       [[2,3,4]],#n=5
       [[3,4,5]],#n=6
       [[3,4,6],[3,5,6]],#n=7
       [[4,5,7],[4,6,7]],#n=8
        [],[],[],[],[],[]]
    ijk=ijka[n]
    
     ### uncomment this block for search through all applications of Bonnesen
    ijk=Combinations(range(n+1),3).list() 
     ###
    
    search=True
    while search:
        
        found=False
        #checking for any improvements by bonnesen
        for indexes in ijk:
            i,j,k=indexes[0],indexes[1],indexes[2]
            bn = bonnesen(i,j,k,w[i],w[k],n)
            if bn<w[j]:
                w[j]=bn
                a = (den/B)*sum(binomial(n,t)*w[t] for t in range(0,n+1))
                #print(a.n(),i,j,k) can be uncommented to see exact applications of Bonnesen
                found=True
        if not found:
            search=False

    #check
    a = (den/B)*sum(binomial(n,t)*w[t] for t in range(0,n+1))
    print("final bound",floor(a.n())) 


n= 2
covering density 1.20919957615615
starting bound 7.20919957615614
final bound 7
n= 3
covering density 1.46350306896682
starting bound 30.6393284494978
final bound 30
n= 4
covering density 1.76552850814935
starting bound 167.969514471425
final bound 167
n= 5
covering density 2.12428590899159
starting bound 1092.61145435622
final bound 933
n= 6
covering density 2.46480600000000
starting bound 7459.19652342020
final bound 6137
n= 7
covering density 2.90002900000000
starting bound 54409.6145773904
final bound 41377
n= 8
covering density 3.14220700000000
starting bound 382480.227466842
final bound 284096
n= 9
covering density 4.34019000000000
starting bound 3.58300372410983e6
final bound 2740831
n= 10
covering density 5.25171360260419
starting bound 3.07074088236695e7
final bound 22106488
n= 11
covering density 5.59834300000000
starting bound 2.41726691594449e8
final bound 184114240
n= 12
covering density 7.51011376957655
starting bound 2.49138907735487e9
final bound 1809629582


In [48]:
### proof of theorem 1: symmetric body
def bonnesen(i,j,k,wi,wk,R):
    #here i<j<k
    return ((k-j)*(i+1)*R^i*wi+(j-i)*(k+1)*R^k*wk)/(R^j*(j+1)*(k-i))

dens = [0]*14
dens[1:5]=[1, 1.209199, 1.463505, 1.765529, 2.124286]
dens[6:13]=[2.464801, 2.900024, 3.142202, 4.340185, 5.251713 ,5.598338, 7.510113, 7.864060] #as in Schurmann and Vallentin
for i in range(14):
    dens[i] += 0.000005 #we upper bound the rounded values for the density from Schurmann and Vallentin; for A_n^* lattice exact is used below
#for n in range(4,7):
for n in range(2,13):
    print("n=",n)
    B=pi^(n/2)/gamma(n/2+1) #|B_2^n|
    Bprev=pi^((n-1)/2)/gamma((n-1)/2+1)
    den=dens[n]
    if n in [2,3,4,5,10,12]:
        den = sqrt(n+1)*(n*(n+2)/12/(n+1))^(n/2)*B #exact value for density A_n^* lattice
    print("covering density", den.n())
    S=2^n #cube volume W_0(C^n)
    w = [S for i in range(n)]+[B]
    w[n-1]=2*Bprev #Follows from Kubota's formula
    a = (den/B)*sum(binomial(n,t)*w[t] for t in range(0,n+1))
    print("starting bound",a.n())

    ###these are sufficient for the result
    ijka=[[],[],[],[],
       [[1,2,3]],#n=4
       [[2,3,4]],#n=5
       [[2,3,5],[2,4,5]],#n=6
          [],[],[],[],[],[],[],[]]
    ijk=ijka[n]
    
     ### uncomment this block for search through all applications of Bonnesen
    ijk=Combinations(range(n+1),3).list() 
     ###
    
    search=True
    while search:
        
        found=False
        #checking for any improvements by bonnesen
        for indexes in ijk:
            i,j,k=indexes[0],indexes[1],indexes[2]
            bn = bonnesen(i,j,k,w[i],w[k],sqrt(n)) #R=sqrt(n) for symmetric
            if bn<w[j]:
                w[j]=bn
                a = (den/B)*sum(binomial(n,t)*w[t] for t in range(0,n+1))
                print(a.n(),i,j,k) #can be uncommented to see exact applications of Bonnesen
                found=True
        if not found:
            search=False

    #check
    a = (den/B)*sum(binomial(n,t)*w[t] for t in range(0,n+1))
    print("final bound",floor(a.n())) 


n= 2
covering density 1.20919957615615
starting bound 5.82800172967315
final bound 5
n= 3
covering density 1.46350306896682
starting bound 19.2296067668164
final bound 19
n= 4
covering density 1.76552850814935
starting bound 76.7222198955196
75.3009838074592 0 2 3
72.0785840654746 1 2 3
final bound 72
n= 5
covering density 2.12428590899159
starting bound 357.806320108065
312.884075333091 0 3 4
307.189420998025 1 3 4
305.987464402981 2 3 4
final bound 305
n= 6
covering density 2.46480600000000
starting bound 1772.55213520375
1708.49019558512 0 3 5
1428.22802117901 0 4 5
1382.16221513098 1 3 4
1359.12931210697 1 3 5
1350.66648315139 1 4 5
1312.07480230871 2 3 4
1299.21090869448 2 3 5
1292.90891360242 2 4 5
final bound 1292
n= 7
covering density 2.90002900000000
starting bound 9475.16954626083
8179.73611094235 0 4 6
6872.89007986470 0 5 6
6794.73564153988 1 4 5
6747.84297854499 1 4 6
6736.02713810708 1 5 6
6574.12478911313 2 3 4
6437.97828836956 2 3 5
6369.90503799778 2 3 6
6287.571373348

In [41]:
# optimized Rogers for n up to 14

def ff(n,x):
    return (1+x)^n*(1+n*log(1/x))

for n in range(2,15):
    #linear exhaustive search over equidistant partition of the interval (0,1/n)
    N=1000 #number of nodes; this parameter may be increased if more accuracy is needed
    left=0
    right=1/n
    minv = Infinity
    for i in range(1,N):
        t = (left*i+right*(N-i))/N
        val = ff(n,t)
        if val<minv:
            minv=val
            argt=t
    f3=minv*binomial(2*n,n)
    f4=minv*2^n
    print("n:",n, "density:", minv.n(),"H_n bound:",floor(f3.n()),"H_n^s bound:",floor(f4.n()))


n: 2 density: 5.37008271805204 H_n bound: 32 H_n^s bound: 21
n: 3 density: 10.0641226122646 H_n bound: 201 H_n^s bound: 80
n: 4 density: 14.9169851018180 H_n bound: 1044 H_n^s bound: 238
n: 5 density: 20.0243588235824 H_n bound: 5046 H_n^s bound: 640
n: 6 density: 25.3627673009858 H_n bound: 23435 H_n^s bound: 1623
n: 7 density: 30.8982924884095 H_n bound: 106042 H_n^s bound: 3954
n: 8 density: 36.6038891485749 H_n bound: 471092 H_n^s bound: 9370
n: 9 density: 42.4585023493765 H_n bound: 2064332 H_n^s bound: 21738
n: 10 density: 48.4455144879283 H_n bound: 8950599 H_n^s bound: 49608
n: 11 density: 54.5515291489705 H_n bound: 38482394 H_n^s bound: 111721
n: 12 density: 60.7655657003799 H_n bound: 164319569 H_n^s bound: 248895
n: 13 density: 67.0784505202403 H_n bound: 697656132 H_n^s bound: 549506
n: 14 density: 73.4824357716896 H_n bound: 2947865482 H_n^s bound: 1203936
